In [2]:
# !git clone https://github.com/fastai/course-v3/
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
%cd /content/course-v3/nbs/dl2/
from exp.nb_05b import *

/content/course-v3/nbs/dl2


In [3]:
def get_data():
    # path = datasets.download_data(MNIST_URL, ext='.gz')
    path = '/content/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [4]:
import inspect; inspect.getsource(normalize)

'def normalize(x, m, s): return (x-m)/s\n'

In [5]:
x_train, y_train, x_valid, y_valid = get_data()
def normalize_to(x_train, x_valid):
    m, s = x_train.mean(), x_train.std()
    return (x_train-m)/s, (x_valid-m)/s
x_train, x_valid = normalize_to(x_train, x_valid)

train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 512
c = y_train.max().item()+1

In [12]:
# cnn
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x): return self.func(x)

def mnist_view(x): return x.view(-1, 1, 28, 28)

def flatten(x): return x.view(x.shape[0], -1)

def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32, data.c)
    )

# cuda
class CudaCallback(Callback):
    def begin_fit(self):
        self.model = self.model.cuda()
    def begin_batch(self):
        self.run.xb, self.run.yb = self.xb.cuda(), self.yb.cuda()    

In [13]:
device = torch.device('cuda', 0)
torch.cuda.set_device(device)

In [14]:
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        CudaCallback]

nfs = [8, 16, 32, 64, 64]
model = get_cnn_model(data)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)
run.fit(3, learn)

train: [1.87353046875, tensor(0.3653, device='cuda:0')]
valid: [1.05581533203125, tensor(0.6204, device='cuda:0')]
train: [0.371524140625, tensor(0.8862, device='cuda:0')]
valid: [0.2044984619140625, tensor(0.9372, device='cuda:0')]
train: [0.1766173828125, tensor(0.9459, device='cuda:0')]
valid: [0.1392503173828125, tensor(0.9591, device='cuda:0')]


---

- A5

In [40]:
xb = next(iter(data.train_dl))[0].view(-1, 1, 28, 28)
model = nn.Sequential(
        Lambda(mnist_view),
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=2, padding=2), nn.ReLU(),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        # Lambda(flatten),
        # nn.Linear(32, data.c)
    )
model(xb).shape

torch.Size([512, 32, 1, 1])

In [37]:
flatten(model(xb)).shape

torch.Size([512, 32])

In [38]:
print(inspect.getsource(flatten))

def flatten(x): return x.view(x.shape[0], -1)



In [18]:
nfs

[8, 16, 32, 64, 64]

In [49]:
def conv2d(ni, nf, ks, stride=2, **kwargs):
    return nn.Sequential(
        nn.Conv2d(ni, nf, ks, stride, padding=ks//2, **kwargs), nn.ReLU()
        )
def get_cnn_layers(nfs, data, **kwargs):
    nfs = [1] + nfs
    return [conv2d(nfs[i], nfs[i+1], 5 if i==0 else 3, **kwargs) for i in range(len(nfs)-1)] + [
              nn.MaxPool2d(1), Lambda(flatten), nn.Linear(nfs[-1], data.c)]

# we need to unpack the list since list is not a module subclass
def get_cnn_model(nfs, data, **kwargs):
    return nn.Sequential(*get_cnn_layers(nfs, data, **kwargs))

In [50]:
get_cnn_model(nfs, data)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (5): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
|